In [26]:
import mdtraj
import numpy as np
file_name = '/home/jiangdapeng/PepGLAD/results/fixseq/references/1bjr_ref.pdb'

# MDtraj
traj = mdtraj.load(file_name)
# SS calculation
pdb_ss = mdtraj.compute_dssp(traj, simplified=True)[0]  # (L, )

ss_count = {
    'alpha': np.sum(pdb_ss == 'H'),
    'beta': np.sum(pdb_ss == 'E'),
}
print("ss_count", ss_count)

ss_count {'alpha': 79, 'beta': 65}


In [17]:
# 获取拓扑对象
topology = traj.topology

# 遍历链并打印编号和链的名称
for chain in topology.chains:
    print(f"链编号: {chain.index}")

链编号: 0
链编号: 1


In [2]:
import mdtraj as md
import numpy as np

file_name = '/home/jiangdapeng/PepGLAD/results/attn_codesign/candidates/1bjr/1bjr_gen_0.pdb'

# Load the PDB file and select chain B
traj = md.load(file_name)
topology = traj.topology
chain_B_atoms = topology.select("chainid == 1")  # 通常 PDB 文件中的链从 0 开始编号，这里假设链B的编号为1

# Slice the trajectory to only include chain B
traj_chain_B = traj.atom_slice(chain_B_atoms)

# Compute secondary structure for chain B
pdb_ss = md.compute_dssp(traj_chain_B, simplified=False)[0]  # (L, )

# Count the secondary structure elements
ss_count = {
    'alpha': np.sum(pdb_ss == 'H'),
    'beta': np.sum(pdb_ss == 'E'),
}
print("ss_count for chain B", ss_count)
print("pdb_ss for chain B", pdb_ss)

ss_count for chain B {'alpha': 0, 'beta': 0}
pdb_ss for chain B [' ' ' ' ' ' 'S' 'S' 'S' ' ' ' ' ' ' ' ']


In [ ]:
import os
import mdtraj as md
# directory = '/home/jiangdapeng/PepGLAD/results/attn_codesign/candidates'
directory = '/home/jiangdapeng/PepGLAD/results/mlp_codesign_1/candidates'
strcture_count = 0
peptide_count = 0

for root, dirs, files in os.walk(directory):
    for file in files:
        file_name = os.path.join(root, file)
        traj = md.load(file_name)
        topology = traj.topology
        chain_B_atoms = topology.select("chainid == 1")  # 通常 PDB 文件中的链从 0 开始编号，这里假设链B的编号为1

        # Slice the trajectory to only include chain B
        traj_chain_B = traj.atom_slice(chain_B_atoms)

        # Compute secondary structure for chain B
        pdb_ss = md.compute_dssp(traj_chain_B, simplified=False)[0]  # (L, )

        # Count the secondary structure elements
        ss_count = {
        'alpha': np.sum(pdb_ss == 'H'),
        'beta': np.sum(pdb_ss == 'E'),
        }
        if ss_count['alpha']!=0:
            print(file_name)
            print("ss_count for chain B", ss_count)
            strcture_count+=ss_count['alpha']
        peptide_count +=1
print(f'average structure{strcture_count/peptide_count}')

/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/1t5z/1t5z_gen_0.pdb
ss_count for chain B {'alpha': 6, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/4rs9/4rs9_gen_0.pdb
ss_count for chain B {'alpha': 8, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/4rs9/4rs9_gen_1.pdb
ss_count for chain B {'alpha': 4, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/3r42/3r42_gen_0.pdb
ss_count for chain B {'alpha': 5, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/5yc2/5yc2_gen_1.pdb
ss_count for chain B {'alpha': 4, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/3r7g/3r7g_gen_1.pdb
ss_count for chain B {'alpha': 4, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/4cy3/4cy3_gen_1.pdb
ss_count for chain B {'alpha': 5, 'beta': 0}
/home/jiangdapeng/PepGLAD/results/mlp_codesign_20/candidates/2f31/2f31_gen_0.pdb
ss_count for chain B {'alpha': 6, 'be

In [7]:
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("/data/private/jdp/scibert")
tokz = AutoTokenizer.from_pretrained("/data/private/jdp/scibert")

output_list = []
for i in range(0,40):
    inputs = tokz(f"The peptide has {i} alpha helices.", return_tensors="pt")
    outputs = model(**inputs)
    output_list.append(outputs['pooler_output'])

/data/private/jdp/envs/PepGLAD/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [16]:
embeddings_tensor.cpu().detach().numpy().shape

(20, 1, 768)

In [19]:
import torch
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# 假设你有一个包含20个torch tensor的列表，维度为 (embedding_dim,)
embeddings_tensor = torch.cat(output_list,dim=0)  # 转换为矩阵 (20, embedding_dim)

# # 使用PCA降维到2D
# pca = PCA(n_components=2)
# embeddings_2d = pca.fit_transform(embeddings_tensor.cpu().detach().numpy())

# 如果想使用t-SNE，也可以使用以下代码代替PCA：
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings_tensor.cpu().detach().numpy())

# 绘制散点图
plt.figure(figsize=(10, 8))
for i in range(20):
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], label=f"{i+1}子结构")
    plt.text(embeddings_2d[i, 0], embeddings_2d[i, 1], f"{i+1}", fontsize=9)

plt.title("蛋白质子结构的Embedding分布")
plt.xlabel("降维后的第1主成分")
plt.ylabel("降维后的第2主成分")
plt.legend(title="子结构数量", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

ValueError: perplexity must be less than n_samples

In [ ]:
import mdtraj as md
import numpy as np

file_name = '/home/jiangdapeng/PepGLAD/output/codesign/1ssc_A_B_5.pdb'

# Load the PDB file and select chain B
traj = md.load(file_name)
topology = traj.topology
chain_B_atoms = topology.select("chainid == 1")  # 通常 PDB 文件中的链从 0 开始编号，这里假设链B的编号为1

# Slice the trajectory to only include chain B
traj_chain_B = traj.atom_slice(chain_B_atoms)

# Compute secondary structure for chain B
pdb_ss = md.compute_dssp(traj_chain_B, simplified=False)[0]  # (L, )

# Count the secondary structure elements
ss_count = {
    'alpha': np.sum(pdb_ss == 'H'),
    'beta': np.sum(pdb_ss == 'E'),
}

In [2]:
import torch
trained_model = torch.load('/home/jiangdapeng/PepGLAD/ckpts/LDM_codesign/version_0/checkpoint/epoch399_step21600.ckpt', map_location='cpu',)
model.load_state_dict(trained_model.state_dict(),strict=False)

/data/private/jdp/envs/PepGLAD/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

Prompt_LDMPepDesign(
  (autoencoder): AutoEncoder(
    (aa_feature): SeparatedAminoAcidFeature(
      (aa_embedding): AminoAcidEmbedding(
        (residue_embedding): Embedding(24, 128)
        (res_pos_embedding): Embedding(2048, 128)
        (atom_embedding): Embedding(121, 32)
        (atom_pos_embedding): Embedding(14, 32)
      )
    )
    (encoder): AMEGNN(
      (dropout): Dropout(p=0.1, inplace=False)
      (linear_in): Linear(in_features=128, out_features=128, bias=True)
      (linear_out): Linear(in_features=128, out_features=128, bias=True)
      (gcl_0): AM_E_GCL(
        (dropout): Dropout(p=0.1, inplace=False)
        (edge_mlp): Sequential(
          (0): Linear(in_features=384, out_features=128, bias=True)
          (1): SiLU()
          (2): Linear(in_features=128, out_features=128, bias=True)
          (3): SiLU()
        )
        (radial_linear): Linear(in_features=1024, out_features=128, bias=True)
        (node_mlp): Sequential(
          (0): Linear(in_features=2